In [0]:
%pip install databricks-vectorsearch
dbutils.library.restartPython()

In [0]:
dbutils.widgets.text("catalog_name", "")
catalog_name=dbutils.widgets.get("catalog_name")

In [0]:
vsc_endpoint="databricks_llm_geekcoders"
master_table=f"{catalog_name}.gold.master_dim_patient_info"
id_key="patient_id"
merged_desc_column="merged_desc"
embedding_endpoint="sentence-transformers-all-MiniLM-L6-v2"

In [0]:
%py
sql(f"ALTER TABLE {catalog_name}.`gold`.`master_dim_patient_info` SET TBLPROPERTIES (delta.enableChangeDataFeed = true)")

In [0]:
index_name=master_table+"_index"
print(index_name)

In [0]:
from databricks.vector_search.client import VectorSearchClient

index = None
try: 
  client = VectorSearchClient()
  index = client.create_delta_sync_index(
    endpoint_name=vsc_endpoint,
    source_table_name=master_table,
    index_name=index_name,
    pipeline_type="TRIGGERED",
    primary_key=id_key,
    embedding_source_column=merged_desc_column,
    embedding_model_endpoint_name=embedding_endpoint
  )
  print("Creating new index")
except Exception as e:
  print("Error creating a new index", e)
  index = client.get_index(endpoint_name=vsc_endpoint, index_name="geekcoders_dev.gold.patient_info_index")
  index.sync()

if index:
  index.wait_until_ready()